In [1]:
import os
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Conv2D,MaxPooling2D,Flatten,Dropout,Input
from keras.applications import InceptionV3
from keras.models import Model
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

In [2]:
train_datagen = ImageDataGenerator(rescale=1./255,rotation_range=0.2,shear_range=0.2,zoom_range=0.2,width_shift_range=0.2,
                               height_shift_range=0.2,validation_split=0.2)

batch_size = 8

train_data = train_datagen.flow_from_directory('drowsiness data/train',target_size=(80,80),batch_size=batch_size,
                                               class_mode='binary',subset='training')

validation_data = train_datagen.flow_from_directory('drowsiness data/train',target_size=(80,80),batch_size=batch_size,
                                               class_mode='binary',subset='validation')

Found 16442 images belonging to 2 classes.
Found 4109 images belonging to 2 classes.


In [3]:
test_datagen =ImageDataGenerator(rescale=1./255)
test_data=test_datagen.flow_from_directory('drowsiness data/test',target_size=(80,80),batch_size=batch_size,
                                               class_mode='binary',subset='training')

Found 3129 images belonging to 2 classes.


In [13]:
train_data.image_shape

(80, 80, 3)

In [17]:
model1 = keras.models.load_model("retraining_with_inceptionv3.h5")
model2 = keras.models.load_model("vgg16.h5")
models=[model1,model2]
model_input =keras.Input(shape=(80, 80, 3))
model_outputs = [model(model_input) for model in models]
# print(model_outputs)
ensemble_output = tf.keras.layers.Average()(model_outputs)
ensemble_model = tf.keras.Model(inputs=model_input, outputs=ensemble_output)

In [19]:
p1=ensemble_model.predict(test_data)

392/392 [==============================] - 98s 250ms/step


In [20]:
p2 = model1.predict(test_data)

392/392 [==============================] - 20s 48ms/step


In [21]:
p3 = model2.predict(test_data)

392/392 [==============================] - 87s 222ms/step


In [22]:
# pred=model.predict(test_data)
opn=[]
closen= []
opn2=[]
closen2=[]
opn3=[]
closen3=[]
for p in p1:
    if p[0]>=0.5:
        opn.append(1)
    else:
        closen.append(0)
        
for p in p2:
    if p[0]>=0.5:
        opn2.append(1)
    else:
        closen2.append(0)
        
for p in p3:
    if p[0]>=0.5:
        opn3.append(1)
    else:
        closen3.append(0)
print(len(opn))
print(len(closen))
print(len(opn2))
print(len(closen2))
print(len(opn3))
print(len(closen3))

1197
1932
1213
1916
1079
2050


In [ ]:
def ensemble(models, model_input):
    outputs = [model.outputs[0] for model in models]
    y = layers.Average()(outputs)
    model = Model(model_input, y, name='ensemble')
    return model

In [ ]:
ensemble_model = ensemble([denseNet_model, inception_model], model_input)

In [ ]:
ensemble_model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

In [ ]:
# models = [keras_model, keras_model2]
# model_input = tf.keras.Input(shape=(125, 125, 3))
# model_outputs = [model(model_input) for model in models]
# ensemble_output = tf.keras.layers.Average()(model_outputs)
# ensemble_model = tf.keras.Model(inputs=model_input, outputs=ensemble_output)